In [ ]:
class Elevator:
    def __init__(self, num_floors, start_floor, id, history):
        self.num_floors = num_floors
        self.current_floor = start_floor
        self.commands = []
        self.id = id
        self.state = "standing"
        self.history = history
        self.transition_map = {}
        for floor in range(2, num_floors):
            self.transition_map[floor] = {
                "standing": {
                    "standing": lambda x: x,
                    "moving_up": lambda x: x.move_up,
                    "moving_down": lambda x: x.move_down
                    },
                "open": {
                    "standing": lambda x: x.close_doors,
                    "moving_up": lambda x: x,
                    "moving_down": lambda x: x
                    },
                "moving_up": {
                    "standing": lambda x: x.stop_moving,
                    "moving_up": lambda x: x,
                    "moving_down": lambda x: x.stop_moving
                    },
                "moving_down": {
                    "standing": lambda x: x.stop_moving,
                    "moving_up": lambda x: x.stop_moving,
                    "moving_down": lambda x: x
                    }
            }
        self.transition_map[1] = {
            "standing": {
                "standing": lambda x: x,
                "moving_up": lambda x: x.move_up,
                "moving_down": lambda x: x.move_down
                },
            "open": {
                "standing": lambda x: x.close_doors,
                "moving_up": lambda x: x,
                },
            "moving_up": {
                "standing": lambda x: x.stop_moving,
                "moving_up": lambda x: x,
                },
            "moving_down": {
                "standing": lambda x: x.stop_moving,
                "moving_up": lambda x: x.stop_moving,
                }
            }
        self.transition_map[num_floors] = {
            "standing": {
                "standing": lambda x: x,
                "moving_down": lambda x: x.move_down
                },
            "open": {
                "standing": lambda x: x.close_doors,
                "moving_down": lambda x: x
                },
            "moving_up": {
                "standing": lambda x: x.stop_moving,
                "moving_down": lambda x: x.stop_moving
                },
            "moving_down": {
                "standing": lambda x: x.stop_moving,
                "moving_down": lambda x: x
                }
            }

    def process_floor_request(self, requested_floor):
        try:
            state_transition = self.transition_map[self.current_floor][self.state].get(self.get_transition_state(requested_floor))
            state_transition(self)(requested_floor)
        except:
            raise ValueError("Некорректный номер этажа")

    def get_transition_state(self, requested_floor):
        return "standing"*(self.current_floor==requested_floor) + "moving_up"*(self.current_floor<requested_floor) + "moving_down"*(self.current_floor>requested_floor)

    def open_doors(self):
            self.commands.append("Открытие двери")
            self.history.append(f"{self.id + 1} лифт открыл дври на {self.current_floor} этаже.")
            self.state = "open"

    def close_doors(self):
            self.commands.append("Закрытие дверей")
            self.history.append(f"{self.id + 1} лифт закрыл двери на {self.current_floor} этаже.")
            self.state = "standing"

    def move_up(self, requested_floor):
        self.state = "moving_up"

        while self.current_floor < requested_floor:
            self.current_floor += 1
            self.history.append(f"{self.id + 1} лифт поднялся на этаж. Текущий этаж: {self.current_floor}.")

        self.open_doors()

    def move_down(self, requested_floor):
        self.state = "moving_down"

        while self.current_floor > requested_floor:
            self.current_floor -= 1
            self.history.append(f"{self.id + 1} лифт спустился на этаж ниже. Текущий этаж: {self.current_floor}.")

        self.open_doors()

    def stop_moving(self, requested_floor):
        pass

    def reset(self):
        self.state = "standing"
        self.commands = []

In [ ]:
class ElevatorSystem:
    def __init__(self, num_floors, elevator_info):
        self.history = []
        self.elevators = [Elevator(num_floors, start_floor, i, self.history) for i, start_floor in enumerate(elevator_info)]

    def request(self, call_floor, target_floor):
        free_elevator = min(self.elevators, key=lambda elevator: abs(call_floor-elevator.current_floor))
        free_elevator.process_floor_request(call_floor)
        free_elevator.close_doors()
        free_elevator.process_floor_request(target_floor)
        free_elevator.close_doors()
        self.history.append(f"{free_elevator.id + 1} лифт обслуживает вызов '{call_floor} - {target_floor}'.")

    def run(self, requests):
        for elevator in self.elevators:
            elevator.reset()

        for request in requests:
            call_floor, target_floor = request
            self.request(call_floor, target_floor)

    def get_history(self):
        for step in self.history:
            print(step)

In [ ]:
requests = [(1, 10), (3, 11), (2, 6), (9, 4), (8, 9), (11, 5), (7, 4), (6, 3), (10, 11), (12, 8), (5, 2), (1, 7), (4, 12), (6, 8), (3, 5)]
elevator_system = ElevatorSystem(12, [12, 4])
elevator_system.run(requests)
elevator_system.get_history()

2 лифт спустился на этаж ниже. Текущий этаж: 3.
2 лифт спустился на этаж ниже. Текущий этаж: 2.
2 лифт спустился на этаж ниже. Текущий этаж: 1.
2 лифт открыл дври на 1 этаже.
2 лифт закрыл двери на 1 этаже.
2 лифт поднялся на этаж. Текущий этаж: 2.
2 лифт поднялся на этаж. Текущий этаж: 3.
2 лифт поднялся на этаж. Текущий этаж: 4.
2 лифт поднялся на этаж. Текущий этаж: 5.
2 лифт поднялся на этаж. Текущий этаж: 6.
2 лифт поднялся на этаж. Текущий этаж: 7.
2 лифт поднялся на этаж. Текущий этаж: 8.
2 лифт поднялся на этаж. Текущий этаж: 9.
2 лифт поднялся на этаж. Текущий этаж: 10.
2 лифт открыл дври на 10 этаже.
2 лифт закрыл двери на 10 этаже.
2 лифт обслуживает вызов '1 - 10'.
2 лифт спустился на этаж ниже. Текущий этаж: 9.
2 лифт спустился на этаж ниже. Текущий этаж: 8.
2 лифт спустился на этаж ниже. Текущий этаж: 7.
2 лифт спустился на этаж ниже. Текущий этаж: 6.
2 лифт спустился на этаж ниже. Текущий этаж: 5.
2 лифт спустился на этаж ниже. Текущий этаж: 4.
2 лифт спустился на этаж н